In [6]:
import torch 
import numpy as np
from botorch.models import SingleTaskGP
from botorch  import fit_gpytorch_model

from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.acquisition.monte_carlo import qExpectedImprovement

from botorch.optim import optimize_acqf

from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import norm
from scipy.optimize import minimize
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process.kernels import RBF


In [7]:
f3_inputs = np.load("initial_inputs.npy")
f3_outputs = np.load("initial_outputs.npy")
f3_inputs_2 = np.load("initial_inputs2.npy")
f3_outputs_2 = np.load("initial_outputs2.npy")

In [8]:
f3_inputs = np.concatenate((f3_inputs, f3_inputs_2)) 
f3_inputs = np.concatenate((f3_inputs, np.array([[0.856453, 0.572658, 0.396564]]))) 
f3_inputs = np.concatenate((f3_inputs, np.array([[0.419052, 0.525791, 0.668723]]))) 
f3_inputs = np.concatenate((f3_inputs, np.array([[0.806453, 0.542658, 0.426564]]))) 
f3_inputs = np.concatenate((f3_inputs, np.array([[0.95404 , 0.603434, 0.360707]])))
f3_inputs = np.concatenate((f3_inputs, np.array([[0.378686, 0.252828, 0.414646]]))) 
f3_inputs = np.concatenate((f3_inputs, np.array([[0.872008, 0.592658, 0.394341]]))) 
f3_inputs = np.concatenate((f3_inputs, np.array([[0.876453, 0.552658, 0.376564]]))) 
f3_inputs = np.concatenate((f3_inputs, np.array([[0.836453, 0.579324, 0.385452]]))) 
f3_inputs = np.concatenate((f3_inputs, np.array([[0.867564, 0.561546, 0.416564]]))) 
f3_inputs = np.concatenate((f3_inputs, np.array([[0.852008, 0.563768, 0.409897]]))) 
f3_inputs = np.concatenate((f3_inputs, np.array([[0.840896, 0.583768, 0.429897]]))) 
f3_inputs = np.concatenate((f3_inputs, np.array([[0.756341, 0.548901, 0.444526]]))) 
f3_inputs = np.concatenate((f3_inputs, np.array([[0.79493 , 0.426714, 0.384933]]))) 
f3_inputs = np.concatenate((f3_inputs, np.array([[0.863119, 0.543768, 0.403231]]))) 
f3_inputs = np.concatenate((f3_inputs, np.array([[0.845371, 0.518261, 0.394578]]))) 
f3_inputs = np.concatenate((f3_inputs, np.array([[0.872008, 0.543768, 0.429897]]))) 
f3_inputs = np.concatenate((f3_inputs, np.array([[0.85423 , 0.561545, 0.40323 ]]))) 
f3_inputs = np.concatenate((f3_inputs, np.array([[0.872008, 0.574879, 0.412119]]))) 

f3_outputs = np.concatenate((f3_outputs,f3_outputs_2)) 
f3_outputs = np.append(f3_outputs,-0.007556472239926157)
f3_outputs = np.append(f3_outputs,-0.11117041024766369)
f3_outputs = np.append(f3_outputs,-0.02486221525774178)
f3_outputs = np.append(f3_outputs,-0.02157038795559339)
f3_outputs = np.append(f3_outputs,-0.0503012651579929)
f3_outputs = np.append(f3_outputs,-0.01517794191050481)
f3_outputs = np.append(f3_outputs,-0.01595053598681603)
f3_outputs = np.append(f3_outputs,-0.010925064020108063)
f3_outputs = np.append(f3_outputs,-0.007386906470204272)
f3_outputs = np.append(f3_outputs,-0.004457685991172721)
f3_outputs = np.append(f3_outputs,-0.025865326752195786)
f3_outputs = np.append(f3_outputs,-0.011107628816793764)
f3_outputs = np.append(f3_outputs,-0.01562544524641375)
f3_outputs = np.append(f3_outputs,-0.010331415655822254)
f3_outputs = np.append(f3_outputs,-0.01698559042276884)
f3_outputs = np.append(f3_outputs,-0.017335638304868072)
f3_outputs = np.append(f3_outputs,-0.01188054501063807)
f3_outputs = np.append(f3_outputs,-0.008744864239247193)


In [9]:
variable_value = np.linspace(1, 10, 10)
n_dims = 3 
beta = 0.001
np.random.seed(42)
train_X = torch.tensor(f3_inputs)
train_Y = torch.tensor(f3_outputs).reshape(len(f3_outputs),-1)
best_y = np.max(train_Y.numpy())
best_x = train_X[np.argmax(train_Y.numpy())]
noise_level =0.1
candidate_list = []
best_x_list = []

#train_X = train_X1[0].reshape(8,-1)
#train_Y = train_Y1[0].reshape(1,-1)

for i in range(len(variable_value)):
    noise_level = variable_value[i]
   
    def function_3(x1,x2,x3, noise_level=noise_level):
        return np.sin(5 * x1+4*x2) * (1 - np.tanh(x1 ** 2) - np.tanh(x2)) + \
        np.sin(1 * x3+4*x2) * (1 - np.tanh(x3 ** 2) - np.tanh(x1)) + \
        + np.random.randn() * noise_level
    
    
    gp = SingleTaskGP(train_X, train_Y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)

    #UCB = UpperConfidenceBound(gp, beta=beta)
    EI = qExpectedImprovement(model=gp , best_f=best_y)

    bounds = torch.stack([torch.zeros(n_dims), torch.ones(n_dims)])
    candidate, _  = optimize_acqf(
        EI, bounds=bounds, q=1, num_restarts=50, raw_samples=200,
    )


    x1,x2,x3 = candidate.numpy().ravel().tolist()
    # Get the next value for the function from the script

    y_next = function_3(x1,x2,x3)

    if y_next > best_y:
        best_y = y_next
        best_x = x1,x2,x3


    y_next = torch.tensor(y_next).unsqueeze(-1)

    train_X = torch.cat([train_X,candidate])
    train_Y = torch.cat([train_Y,y_next.unsqueeze(-1)])
    candidate_list.append(candidate) 
    best_x_list.append(best_x)
    #print(f"Next point: {candidate} \n Next Y {y_next}  Best_y, {best_y}")
    #print(f"beta :{beta} noise_level :{noise_level}")
    #print(f"Next point: {candidate} \n Next Y {y_next}")
    #print(f"best_x:{best_x}Best_y,{best_y}")
    

In [10]:
best_x_list

[(1.0, 0.0, 0.4854981601238251),
 (1.0, 0.0, 0.4854981601238251),
 (0.7693067789077759, 0.04371098056435585, 0.290031373500824),
 (0.7316170930862427, 0.0, 0.1812210977077484),
 (0.7316170930862427, 0.0, 0.1812210977077484),
 (0.7316170930862427, 0.0, 0.1812210977077484),
 (0.7099969387054443, 0.0, 0.10540054738521576),
 (0.7099969387054443, 0.0, 0.10540054738521576),
 (0.7099969387054443, 0.0, 0.10540054738521576),
 (0.7099969387054443, 0.0, 0.10540054738521576)]